# Getting the relation between baroclinic streamfunction and cross-stream distance from the SR1b data

In [ ]:
import pickle
import numpy as np
from scipy.io import loadmat
import gsw
import csaps

## Sample points along hydrographic section transect

In [2]:
# read in hydrographic section data
data = loadmat('data/SR1b_section/sr1b_all.mat')

icyc = np.where(data['buf_year'] != 1999)[1] # exclude 1999 # TODO: why?

sr1b_lat = data['buf_lat'][0,icyc]
sr1b_lon = data['buf_lon'][0,icyc]
sr1b_sal = data['buf_sal'][:,icyc]
sr1b_temp = data['buf_temp'][:,icyc]
sr1b_press = data['buf_press'].reshape(3000)
sr1b_year = data['buf_year'][:,icyc].reshape(316)

In [3]:
# calculate absolute salinity and conservative temperature
sr1b_SA = gsw.SA_from_SP(sr1b_sal, sr1b_press.reshape(3000,1), sr1b_lon.reshape(1,316), sr1b_lat.reshape(1,316)) # absolute salinity
sr1b_CT = gsw.conversions.CT_from_t(sr1b_SA, sr1b_temp, sr1b_press.reshape(3000,1)) # conservative temperature

# compute baroclinic streamfunction
gpan_sr1b = -gsw.geo_strf_dyn_height(sr1b_SA, sr1b_CT, sr1b_press, 1500) #geopotential height anomaly / dynamic height anomaly
sr1b_phi1500 = abs(gpan_sr1b[249,:]) # index 250 is at 500 dbar

# calculate distance from southern end of section
sr1b_dist = np.zeros(len(sr1b_lat))
for i in range(len(sr1b_lat)):
  sr1b_dist[i] = gsw.distance(np.mean(sr1b_lon)*np.ones(2), np.array([min(sr1b_lat), sr1b_lat[i]]))[0]

# Calculate mean relationship between phi1500 and distance from southern edge of section: fit a smoothing spline through relationship
pp_dist_sr1b = csaps.csaps(sr1b_phi1500[np.argsort(sr1b_phi1500)[:-np.count_nonzero(np.isnan(sr1b_phi1500))]],
                      sr1b_dist[np.argsort(sr1b_phi1500)[:-np.count_nonzero(np.isnan(sr1b_phi1500))]],
                      smooth = 0.85)

# save the spline fit
with open('data/SR1b_section/pp_dist_sr1b.pkl', 'wb') as f:
    pickle.dump(pp_dist_sr1b, f)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)
